In [2]:
import pandas as pd
import requests
from newspaper import Article

In [28]:
def get_article_info(url):
    article = Article(url)
    article.download()
    article.parse()
    
    article_details = {'title'       : article.title,
                       'text'        : article.text,
                       'webUrl'      : article.url,
                       'authors'     : article.authors,
                       'html'        : article.html,
                       'date'        : article.publish_date,
                       'description' : article.meta_description}
    return article_details

In [29]:
def get_guardian(page_number):
    params = { 'q' : 'Brexit',
               'api-key' : '283a6176-f92c-46ec-91ce-29efdb59ffab',
              'page' : page_number,
              'page-size' : 50 }
    j = requests.get('https://content.guardianapis.com/search', params=params)
    return pd.DataFrame(j.json()['response']['results'])

In [30]:
metas = []

for page in range(1,10):
    metas.append(get_guardian(page))
    

In [31]:
meta_df = pd.concat(metas)

In [32]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 450 entries, 0 to 49
Data columns (total 11 columns):
apiUrl                450 non-null object
id                    450 non-null object
isHosted              450 non-null bool
pillarId              449 non-null object
pillarName            449 non-null object
sectionId             450 non-null object
sectionName           450 non-null object
type                  450 non-null object
webPublicationDate    450 non-null object
webTitle              450 non-null object
webUrl                450 non-null object
dtypes: bool(1), object(10)
memory usage: 39.1+ KB


In [46]:
 meta_df['webUrl'].values[:5]

array(['https://www.theguardian.com/politics/2018/jul/09/brexit-developments-or-not',
       'https://www.theguardian.com/politics/2018/may/29/should-labour-oppose-brexit',
       'https://www.theguardian.com/world/2018/jul/30/monday-briefing-thinktank-in-brexit-influencing-game',
       'https://www.theguardian.com/politics/ng-interactive/2017/jul/20/where-are-we-up-to-in-these-brexit-talks',
       'https://www.theguardian.com/tv-and-radio/2018/jul/15/radio-podcasts-politics-brexit-remainiacs-ed-miliband'],
      dtype=object)

In [39]:
articles = []

for url in meta_df['webUrl'].values[:]:
    articles.append(get_article_info(url))

KeyboardInterrupt: 

In [40]:
article_db = pd.DataFrame(articles)

In [47]:
article_db.head()

,authors,date,description,html,text,title,webUrl
0,[],2018-07-09,Letters: Readers respond to the government’s l...,"\n<!DOCTYPE html>\n<html id=""js-context"" class...",Your editorial (Reality is dawning for the PM ...,Brexit developments – or not | Letters,https://www.theguardian.com/politics/2018/jul/...
1,[],2018-05-29,Letters: John Cookson says there is no politic...,"\n<!DOCTYPE html>\n<html id=""js-context"" class...",The situation of leave voters in Labour consti...,Should Labour oppose Brexit? | Letters,https://www.theguardian.com/politics/2018/may/...
2,[Alison Rourke],2018-07-30,Institute of Economic Affairs offers ministeri...,"\n<!DOCTYPE html>\n<html id=""js-context"" class...",Top story: IEA denies ‘cash for access’ system...,Monday briefing: thinktank in ‘Brexit-influenc...,https://www.theguardian.com/world/2018/jul/30/...
3,[Seán Clarke],2017-07-20,"After two years, Britain finally produced its ...","\n<!DOCTYPE html>\n<html id=""js-context"" class...",Channel 8 of 8\n\nOther areas of cooperation 📚...,Where do the Brexit negotiations stand?,https://www.theguardian.com/politics/ng-intera...
4,[Miranda Sawyer],2018-07-15,The Brexit podcasters got giddy in the wake of...,"\n<!DOCTYPE html>\n<html id=""js-context"" class...",Remainiacs | Audioboom\n\nBrexitcast | Radio 5...,The week in radio and podcasts: Remainiacs; Br...,https://www.theguardian.com/tv-and-radio/2018/...


In [42]:
merged_df = meta_df.merge(article_db, left_on='webUrl', right_on='webUrl')

In [49]:
merged_df.head()

,apiUrl,id,isHosted,pillarId,pillarName,sectionId,sectionName,type,webPublicationDate,webTitle,webUrl,authors,date,description,html,text,title
0,https://content.guardianapis.com/politics/2018...,politics/2018/jul/09/brexit-developments-or-not,False,pillar/news,News,politics,Politics,article,2018-07-09T16:57:33Z,Brexit developments – or not | Letters,https://www.theguardian.com/politics/2018/jul/...,[],2018-07-09,Letters: Readers respond to the government’s l...,"\n<!DOCTYPE html>\n<html id=""js-context"" class...",Your editorial (Reality is dawning for the PM ...,Brexit developments – or not | Letters
1,https://content.guardianapis.com/politics/2018...,politics/2018/may/29/should-labour-oppose-brexit,False,pillar/news,News,politics,Politics,article,2018-05-29T17:23:29Z,Should Labour oppose Brexit? | Letters,https://www.theguardian.com/politics/2018/may/...,[],2018-05-29,Letters: John Cookson says there is no politic...,"\n<!DOCTYPE html>\n<html id=""js-context"" class...",The situation of leave voters in Labour consti...,Should Labour oppose Brexit? | Letters
2,https://content.guardianapis.com/world/2018/ju...,world/2018/jul/30/monday-briefing-thinktank-in...,False,pillar/news,News,world,World news,article,2018-07-30T05:31:56Z,Monday briefing: thinktank in ‘Brexit-influenc...,https://www.theguardian.com/world/2018/jul/30/...,[Alison Rourke],2018-07-30,Institute of Economic Affairs offers ministeri...,"\n<!DOCTYPE html>\n<html id=""js-context"" class...",Top story: IEA denies ‘cash for access’ system...,Monday briefing: thinktank in ‘Brexit-influenc...
3,https://content.guardianapis.com/politics/ng-i...,politics/ng-interactive/2017/jul/20/where-are-...,False,pillar/news,News,politics,Politics,interactive,2018-07-27T08:43:14Z,Where do the Brexit negotiations stand?,https://www.theguardian.com/politics/ng-intera...,[Seán Clarke],2017-07-20,"After two years, Britain finally produced its ...","\n<!DOCTYPE html>\n<html id=""js-context"" class...",Channel 8 of 8\n\nOther areas of cooperation 📚...,Where do the Brexit negotiations stand?
4,https://content.guardianapis.com/tv-and-radio/...,tv-and-radio/2018/jul/15/radio-podcasts-politi...,False,pillar/arts,Arts,tv-and-radio,Television & radio,article,2018-07-15T06:00:02Z,The week in radio and podcasts: Remainiacs; Br...,https://www.theguardian.com/tv-and-radio/2018/...,[Miranda Sawyer],2018-07-15,The Brexit podcasters got giddy in the wake of...,"\n<!DOCTYPE html>\n<html id=""js-context"" class...",Remainiacs | Audioboom\n\nBrexitcast | Radio 5...,The week in radio and podcasts: Remainiacs; Br...
